## Financial Data Preprocessing 

In [126]:
#This Jupyter Notebook is designed to preprocess financial data for predictive analytics. 
#It involves cleaning, transforming, and integrating datasets to ensure they are ready for machine learning models. 
#The pipeline employs Python libraries like Pandas, NumPy, and scikit-learn to handle missing values, validate data consistency.
import pandas as pd
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [107]:
pwd

'C:\\Users\\dishi\\Financial Data Preprocessing Pipeline'

### Loading the datasets into Pandas DataFrames

In [108]:
# File paths
annual_pl_path = "Annual_P_L_1_final.csv"
balance_sheet_path = "Balance_Sheet_final.csv"
cash_flow_path = "cash_flow_statments_final.csv"
other_metrics_path = "other_metrics_final.csv"
ratios_path = "ratios_1_final.csv"

try:
    annual_pl_df = pd.read_csv(annual_pl_path)
    balance_sheet_df = pd.read_csv(balance_sheet_path)
    cash_flow_df = pd.read_csv(cash_flow_path)
    other_metrics_df = pd.read_csv(other_metrics_path)
    ratios_df = pd.read_csv(ratios_path)
    
    # Display first few rows of each DataFrame
    print("Annual Profit & Loss DataFrame:\n", annual_pl_df.head(), "\n")
    print("Balance Sheet DataFrame:\n", balance_sheet_df.head(), "\n")
    print("Cash Flow Statements DataFrame:\n", cash_flow_df.head(), "\n")
    print("Other Metrics DataFrame:\n", other_metrics_df.head(), "\n")
    print("Ratios DataFrame:\n", ratios_df.head(), "\n")
    
except FileNotFoundError as e:
    print("Error loading files. Please check the file paths.")
    print(e)
except Exception as e:
    print("An unexpected error occurred:")
    print(e)

Annual Profit & Loss DataFrame:
                Name  BSE Code    NSE Code  \
0        20 Microns  533022.0   20MICRONS   
1  21st Cent. Mgmt.  526921.0  21STCENMGM   
2           360 ONE  542772.0      360ONE   
3       3B Blackbio  532067.0      __NA__   
4   3C IT Solutions  544190.0      __NA__   

                                Industry  Current Price    Sales    OPM  \
0             Mining / Minerals / Metals         224.55   777.49  13.59   
1                  Finance & Investments          70.44    34.92  94.27   
2                  Finance & Investments        1009.40  2920.91  58.37   
3                             Healthcare        1182.00    74.12  44.50   
4  Computers - Software - Medium / Small          44.01    61.93   5.17   

   Profit after tax  Return on capital employed    EPS  ...  \
0             57.38                       21.70  15.89  ...   
1             32.23                       73.00  30.70  ...   
2            804.18                       14.47  22.41  

In [109]:
columns_name = annual_pl_df.columns
print(columns_name)

Index(['Name', 'BSE Code', 'NSE Code', 'Industry', 'Current Price', 'Sales',
       'OPM', 'Profit after tax', 'Return on capital employed', 'EPS',
       'Change in promoter holding', 'Sales last year',
       'Operating profit last year', 'Other income last year',
       'EBIDT last year', 'Depreciation last year', 'EBIT last year',
       'Interest last year', 'Profit before tax last year', 'Tax last year',
       'Profit after tax last year', 'Extraordinary items last year',
       'Net Profit last year', 'Dividend last year', 'Material cost last year',
       'Employee cost last year', 'OPM last year', 'NPM last year',
       'Operating profit', 'Interest', 'Depreciation', 'EPS last year', 'EBIT',
       'Net profit', 'Current Tax', 'Tax', 'Other income',
       'Last annual result date', 'Sales preceding year',
       'Operating profit preceding year', 'Other income preceding year',
       'EBIDT preceding year', 'Depreciation preceding year',
       'EBIT preceding year', 'Inter

In [110]:
# Remove the "Current Price" and "Market Capitalization" columns
def drop_unnecessary_columns(df, columns_to_drop, file_name):
    try:
        df = df.drop(columns=columns_to_drop, errors='ignore')
        print(f"Columns {columns_to_drop} dropped from {file_name}.")
    except Exception as e:
        print(f"An error occurred while dropping columns from {file_name}: {e}")
    return df

# Columns to drop
columns_to_remove = ["Current Price", "Market Capitalization"]

# Apply the function to the DataFrames
annual_pl_df = drop_unnecessary_columns(annual_pl_df, columns_to_remove, "Annual_P_L_1_final.csv")
balance_sheet_df = drop_unnecessary_columns(balance_sheet_df, columns_to_remove, "Balance_Sheet_final.csv")
cash_flow_df = drop_unnecessary_columns(cash_flow_df, columns_to_remove, "cash_flow_statments_final.csv")
ratios_df = drop_unnecessary_columns(ratios_df, columns_to_remove, "ratios_1_final.csv")

Columns ['Current Price', 'Market Capitalization'] dropped from Annual_P_L_1_final.csv.
Columns ['Current Price', 'Market Capitalization'] dropped from Balance_Sheet_final.csv.
Columns ['Current Price', 'Market Capitalization'] dropped from cash_flow_statments_final.csv.
Columns ['Current Price', 'Market Capitalization'] dropped from ratios_1_final.csv.


In [111]:
def check_columns_existence(df, columns, df_name="DataFrame"):
    """
    Check if specified columns exist in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to check.
        columns (list): List of column names to check.
        df_name (str): Name of the DataFrame (for logging).
    
    Returns:
        dict: Dictionary with column names as keys and True/False as values indicating existence.
    """
    result = {col: (col in df.columns) for col in columns}
    print(f"\nColumn existence check for {df_name}:")
    for col, exists in result.items():
        print(f"  - {col}: {'Exists' if exists else 'Does not exist'}")
    return result

# Example usage
columns_to_check = ["Current Price", "Market Capitalization"]
check_columns_existence(annual_pl_df, columns_to_check, "Annual Profit & Loss")
check_columns_existence(balance_sheet_df, columns_to_check, "Balance Sheet")
check_columns_existence(other_metrics_df, columns_to_check, "Other Metrics")


Column existence check for Annual Profit & Loss:
  - Current Price: Does not exist
  - Market Capitalization: Does not exist

Column existence check for Balance Sheet:
  - Current Price: Does not exist
  - Market Capitalization: Does not exist

Column existence check for Other Metrics:
  - Current Price: Exists
  - Market Capitalization: Exists


{'Current Price': True, 'Market Capitalization': True}

In [112]:
def handle_missing_values(df_dict, drop_threshold=0.5, impute_strategy="mean"):
    """
    Handle missing values for multiple DataFrames.

    Parameters:
        df_dict (dict): Dictionary where keys are DataFrame names, and values are the DataFrames.
        drop_threshold (float): Threshold for dropping columns (proportion of missing values).
        impute_strategy (str): Strategy for imputing missing values ("mean", "median", or "mode").

    Returns:
        dict: Dictionary of DataFrames with missing values handled.
    """
    processed_dfs = {}

    for name, df in df_dict.items():
        print(f"\nHandling missing values for {name}:")
        
        # Drop columns with excessive missing data
        missing_proportion = df.isnull().mean()
        cols_to_drop = missing_proportion[missing_proportion > drop_threshold].index
        df = df.drop(columns=cols_to_drop)
        print(f"Dropped columns with >{drop_threshold*100}% missing values: {list(cols_to_drop)}")
        
        # Impute missing values for numeric columns
        numeric_cols = df.select_dtypes(include=["number"]).columns
        if impute_strategy == "mean":
            df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
        elif impute_strategy == "median":
            df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
        
        # Impute missing values for categorical columns
        categorical_cols = df.select_dtypes(include=["object", "category"]).columns
        for col in categorical_cols:
            df[col] = df[col].fillna(df[col].mode()[0])
        
        print(f"Imputed missing values using '{impute_strategy}' for numeric columns.")
        print(f"Imputed missing values using mode for categorical columns.")
        
        # Add the processed DataFrame back to the dictionary
        processed_dfs[name] = df
    
    return processed_dfs

# Create a dictionary of DataFrames
dataframes = {
    "Annual Profit & Loss": annual_pl_df,
    "Balance Sheet": balance_sheet_df,
    "Cash Flow Statements": cash_flow_df,
    "Other Metrics": other_metrics_df,
    "Ratios": ratios_df
}

# Apply the function to all DataFrames collectively
processed_dataframes = handle_missing_values(dataframes, drop_threshold=0.5, impute_strategy="mean")

# Extract processed DataFrames from the result dictionary
annual_pl_df = processed_dataframes["Annual Profit & Loss"]
balance_sheet_df = processed_dataframes["Balance Sheet"]
cash_flow_df = processed_dataframes["Cash Flow Statements"]
other_metrics_df = processed_dataframes["Other Metrics"]
ratios_df = processed_dataframes["Ratios"]


Handling missing values for Annual Profit & Loss:
Dropped columns with >50.0% missing values: []
Imputed missing values using 'mean' for numeric columns.
Imputed missing values using mode for categorical columns.

Handling missing values for Balance Sheet:
Dropped columns with >50.0% missing values: []
Imputed missing values using 'mean' for numeric columns.
Imputed missing values using mode for categorical columns.

Handling missing values for Cash Flow Statements:
Dropped columns with >50.0% missing values: []
Imputed missing values using 'mean' for numeric columns.
Imputed missing values using mode for categorical columns.

Handling missing values for Other Metrics:
Dropped columns with >50.0% missing values: []
Imputed missing values using 'mean' for numeric columns.
Imputed missing values using mode for categorical columns.

Handling missing values for Ratios:
Dropped columns with >50.0% missing values: ['Credit rating']
Imputed missing values using 'mean' for numeric columns.
Im

In [113]:
columns_to_check = ["Credit Rating"]
check_columns_existence(ratios_df, columns_to_check, "Ratios DF")


Column existence check for Ratios DF:
  - Credit Rating: Does not exist


{'Credit Rating': False}

In [114]:
def remove_duplicates_from_dfs(df_dict, subset=None, keep="first"):
    """
    Remove duplicate rows from multiple DataFrames.

    Parameters:
        df_dict (dict): Dictionary of DataFrames where keys are DataFrame names and values are the DataFrames.
        subset (list or None): List of columns to check for duplicates. If None, all columns are used.
        keep (str): Determines which duplicates to keep:
                    - "first" : Keeps the first occurrence (default).
                    - "last"  : Keeps the last occurrence.
                    - False   : Removes all duplicates.

    Returns:
        dict: Dictionary of cleaned DataFrames without duplicates.
    """
    cleaned_dfs = {}
    
    for name, df in df_dict.items():
        print(f"\nRemoving duplicates in '{name}':")
        
        # Check duplicates before removal
        duplicate_count = df.duplicated(subset=subset, keep=keep).sum()
        print(f"Number of duplicate rows: {duplicate_count}")
        
        # Remove duplicates
        cleaned_df = df.drop_duplicates(subset=subset, keep=keep).reset_index(drop=True)
        cleaned_dfs[name] = cleaned_df
        
        print(f"Duplicates removed. New shape: {cleaned_df.shape}")
    
    return cleaned_dfs

# Call the function on all DataFrames
cleaned_dataframes = remove_duplicates_from_dfs(processed_dataframes)


Removing duplicates in 'Annual Profit & Loss':
Number of duplicate rows: 0
Duplicates removed. New shape: (4668, 56)

Removing duplicates in 'Balance Sheet':
Number of duplicate rows: 0
Duplicates removed. New shape: (4668, 49)

Removing duplicates in 'Cash Flow Statements':
Number of duplicate rows: 0
Duplicates removed. New shape: (4668, 34)

Removing duplicates in 'Other Metrics':
Number of duplicate rows: 0
Duplicates removed. New shape: (4668, 44)

Removing duplicates in 'Ratios':
Number of duplicate rows: 0
Duplicates removed. New shape: (4668, 47)


In [115]:
def round_numeric_columns(df):
    """
    Rounds numeric columns in a DataFrame to two decimal places if they contain more than two decimals.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: DataFrame with numeric columns rounded to two decimals.
    """
    for col in df.select_dtypes(include=[np.number]).columns:
        # Check if column has any values with more than 2 decimals
        if (df[col] % 1).apply(lambda x: len(str(x).split('.')[-1]) > 2).any():
            df[col] = df[col].round(2)
            print(f"Rounded column: {col}")
    return df

# Apply to all processed DataFrames
for name, df in processed_dataframes.items():
    processed_dataframes[name] = round_numeric_columns(df)

# Example: Print one DataFrame to verify
for name, df in processed_dataframes.items():
    print(f"\nSample data from '{name}' after rounding:")
    print(df.head())

Rounded column: Sales
Rounded column: OPM
Rounded column: Profit after tax
Rounded column: Return on capital employed
Rounded column: EPS
Rounded column: Change in promoter holding
Rounded column: Sales last year
Rounded column: Operating profit last year
Rounded column: Other income last year
Rounded column: EBIDT last year
Rounded column: Depreciation last year
Rounded column: EBIT last year
Rounded column: Interest last year
Rounded column: Profit before tax last year
Rounded column: Tax last year
Rounded column: Profit after tax last year
Rounded column: Extraordinary items last year
Rounded column: Net Profit last year
Rounded column: Dividend last year
Rounded column: Material cost last year
Rounded column: Employee cost last year
Rounded column: OPM last year
Rounded column: NPM last year
Rounded column: Operating profit
Rounded column: Interest
Rounded column: Depreciation
Rounded column: EPS last year
Rounded column: EBIT
Rounded column: Net profit
Rounded column: Current Tax


In [116]:
def validate_numeric_data(df_dict, range_checks=None):
    """
    Validate numeric columns for consistency in multiple DataFrames.

    Parameters:
        df_dict (dict): Dictionary of DataFrames where keys are DataFrame names, and values are the DataFrames.
        range_checks (dict): Dictionary specifying range rules for numeric columns 
                             (e.g., {"column_name": {"min": 0, "max": 1000}}).

    Returns:
        dict: Dictionary of DataFrames with invalid rows flagged or removed.
    """
    validated_dfs = {}
    flagged_rows = {}

    for name, df in df_dict.items():
        print(f"\nValidating numeric data in {name}:")
        
        # Initialize container for invalid rows
        invalid_rows = pd.DataFrame()

        # Perform range checks if specified
        if range_checks:
            for col, limits in range_checks.items():
                if col in df.columns:
                    # Validate against the range
                    invalid_mask = (
                        (df[col] < limits["min"]) | (df[col] > limits["max"])
                    )
                    if invalid_mask.any():
                        print(f"Column '{col}' has values outside the range {limits}.")
                        # Collect invalid rows
                        invalid_rows = pd.concat([invalid_rows, df[invalid_mask]])
                        # Optional: Replace invalid values with NaN
                        df.loc[invalid_mask, col] = pd.NA
        
        # Flag dataframes and log issues
        if not invalid_rows.empty:
            flagged_rows[name] = invalid_rows
            print(f"Flagged {len(invalid_rows)} rows for review in {name}.")
        
        # Add validated DataFrame to the result
        validated_dfs[name] = df

    return validated_dfs, flagged_rows

# Define range checks (e.g., no negative values in financial columns)
range_rules = {
    "Revenue": {"min": 0, "max": None},
    "Profit": {"min": None, "max": None},  # No specific range but can add rules later
    "Expenses": {"min": 0, "max": None}
}

# Apply the validation function to all DataFrames
validated_dataframes, flagged_issues = validate_numeric_data(processed_dataframes, range_checks=range_rules)

# Example: Check flagged rows
for df_name, flagged in flagged_issues.items():
    print(f"\nFlagged rows in {df_name}:\n{flagged}")


Validating numeric data in Annual Profit & Loss:

Validating numeric data in Balance Sheet:

Validating numeric data in Cash Flow Statements:

Validating numeric data in Other Metrics:

Validating numeric data in Ratios:


In [117]:
def discretize_dataframes_with_kmeans(df_dict, n_clusters=3):
    """
    Apply clustering-based discretization using KMeans on numeric columns of multiple DataFrames.

    Parameters:
        df_dict (dict): Dictionary of DataFrames where keys are DataFrame names, and values are DataFrames.
        n_clusters (int): Number of clusters (discrete bins) to create for each numeric column.

    Returns:
        dict: Dictionary of DataFrames with discretized numeric columns.
    """
    discretized_dfs = {}
    
    for name, df in df_dict.items():
        print(f"\nProcessing '{name}' for discretization:")
        
        # Copy the DataFrame to avoid overwriting the original data
        discretized_df = df.copy()
        
        # Identify numeric columns
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        
        discretized_columns = []  # To keep track of discretized columns
        
        # Apply KMeans clustering to each numeric column
        for col in numeric_columns:
            # Drop rows with NaN to ensure proper clustering
            col_data = df[col].dropna().values.reshape(-1, 1)
            
            if len(col_data) > n_clusters:  # Apply KMeans only if sufficient data exists
                kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
                cluster_labels = kmeans.fit_predict(col_data)
                
                # Map cluster labels back to the DataFrame
                discretized_df.loc[~df[col].isna(), col] = cluster_labels
                discretized_columns.append(col)
        
        # Log discretized columns
        if discretized_columns:
            print(f"Discretized columns: {discretized_columns}")
        else:
            print("No columns discretized.")
        
        # Save the discretized DataFrame
        discretized_dfs[name] = discretized_df
    
    return discretized_dfs

# Apply the function to all DataFrames
n_clusters = 3  # Number of bins for discretization
discretized_dataframes = discretize_dataframes_with_kmeans(processed_dataframes, n_clusters=n_clusters)

# Example: Check discretized data for one DataFrame
#for df_name, df in discretized_dataframes.items():
 #   print(f"\nSample rows from discretized '{df_name}':")
  #  print(df.head())


Processing 'Annual Profit & Loss' for discretization:
Discretized columns: ['BSE Code', 'Sales', 'OPM', 'Profit after tax', 'Return on capital employed', 'EPS', 'Change in promoter holding', 'Sales last year', 'Operating profit last year', 'Other income last year', 'EBIDT last year', 'Depreciation last year', 'EBIT last year', 'Interest last year', 'Profit before tax last year', 'Tax last year', 'Profit after tax last year', 'Extraordinary items last year', 'Net Profit last year', 'Dividend last year', 'Material cost last year', 'Employee cost last year', 'OPM last year', 'NPM last year', 'Operating profit', 'Interest', 'Depreciation', 'EPS last year', 'EBIT', 'Net profit', 'Current Tax', 'Tax', 'Other income', 'Last annual result date', 'Sales preceding year', 'Operating profit preceding year', 'Other income preceding year', 'EBIDT preceding year', 'Depreciation preceding year', 'EBIT preceding year', 'Interest preceding year', 'Profit before tax preceding year', 'Tax preceding year'

In [118]:
# Loop through each DataFrame in the dictionary and print its content
for df_name, df in discretized_dataframes.items():
    print(f"\nProcessed DataFrame: '{df_name}'")
    print(f"Shape: {df.shape}")  # Display the shape (rows, columns)


Processed DataFrame: 'Annual Profit & Loss'
Shape: (4668, 56)

Processed DataFrame: 'Balance Sheet'
Shape: (4668, 49)

Processed DataFrame: 'Cash Flow Statements'
Shape: (4668, 34)

Processed DataFrame: 'Other Metrics'
Shape: (4668, 44)

Processed DataFrame: 'Ratios'
Shape: (4668, 47)


In [119]:
# Rename columns for each DataFrame except the 'join_key'
processed_dataframes['Annual Profit & Loss'] = processed_dataframes['Annual Profit & Loss'].rename(
    lambda x: x + "_pnl" if x != "join_key" else x, axis=1
)
processed_dataframes['Balance Sheet'] = processed_dataframes['Balance Sheet'].rename(
    lambda x: x + "_bs" if x != "join_key" else x, axis=1
)
processed_dataframes['Cash Flow Statements'] = processed_dataframes['Cash Flow Statements'].rename(
    lambda x: x + "_cf" if x != "join_key" else x, axis=1
)
processed_dataframes['Other Metrics'] = processed_dataframes['Other Metrics'].rename(
    lambda x: x + "_metrics" if x != "join_key" else x, axis=1
)
processed_dataframes['Ratios'] = processed_dataframes['Ratios'].rename(
    lambda x: x + "_ratios" if x != "join_key" else x, axis=1
)

In [120]:
# Merge all processed DataFrames on 'join_key'
final_df = (
    processed_dataframes['Annual Profit & Loss']
    .merge(processed_dataframes['Balance Sheet'], on="join_key", how="inner")
    .merge(processed_dataframes['Cash Flow Statements'], on="join_key", how="inner")
    .merge(processed_dataframes['Other Metrics'], on="join_key", how="inner")
    .merge(processed_dataframes['Ratios'], on="join_key", how="inner")
)

# Display final DataFrame details
print("Final combined DataFrame shape:", final_df.shape)
print(final_df.head())


Final combined DataFrame shape: (4668, 226)
           Name_pnl  BSE Code_pnl NSE Code_pnl  \
0        20 Microns      533022.0    20MICRONS   
1  21st Cent. Mgmt.      526921.0   21STCENMGM   
2           360 ONE      542772.0       360ONE   
3       3B Blackbio      532067.0       __NA__   
4   3C IT Solutions      544190.0       __NA__   

                            Industry_pnl  Sales_pnl  OPM_pnl  \
0             Mining / Minerals / Metals     777.49    13.59   
1                  Finance & Investments      34.92    94.27   
2                  Finance & Investments    2920.91    58.37   
3                             Healthcare      74.12    44.50   
4  Computers - Software - Medium / Small      61.93     5.17   

   Profit after tax_pnl  Return on capital employed_pnl  EPS_pnl  \
0                 57.38                           21.70    15.89   
1                 32.23                           73.00    30.70   
2                804.18                           14.47    22.41  

In [121]:
print("Remaining missing values:")
print(final_df.isnull().sum())

# Verify the data types
print("Data types after preprocessing:")
print(final_df.dtypes)

Remaining missing values:
Name_pnl                                            0
BSE Code_pnl                                        0
NSE Code_pnl                                        0
Industry_pnl                                        0
Sales_pnl                                           0
                                                   ..
Book value preceding year_ratios                    0
Return on capital employed preceding year_ratios    0
Return on assets preceding year_ratios              0
Return on equity preceding year_ratios              0
Number of Shareholders preceding quarter_ratios     0
Length: 226, dtype: int64
Data types after preprocessing:
Name_pnl                                             object
BSE Code_pnl                                        float64
NSE Code_pnl                                         object
Industry_pnl                                         object
Sales_pnl                                           float64
                      

In [122]:
# Apply MinMax scaling to numeric columns
scaler = MinMaxScaler()
numeric_columns = final_df.select_dtypes(include=["float64", "int64"]).columns
final_df[numeric_columns] = scaler.fit_transform(final_df[numeric_columns])

print("Data after scaling:")
print(final_df.head())

Data after scaling:
           Name_pnl  BSE Code_pnl NSE Code_pnl  \
0        20 Microns      0.903222    20MICRONS   
1  21st Cent. Mgmt.      0.892884   21STCENMGM   
2           360 ONE      0.919744       360ONE   
3       3B Blackbio      0.901604       __NA__   
4   3C IT Solutions      0.922147       __NA__   

                            Industry_pnl  Sales_pnl   OPM_pnl  \
0             Mining / Minerals / Metals   0.000885  0.970909   
1                  Finance & Investments   0.000061  0.971671   
2                  Finance & Investments   0.003264  0.971332   
3                             Healthcare   0.000105  0.971201   
4  Computers - Software - Medium / Small   0.000091  0.970830   

   Profit after tax_pnl  Return on capital employed_pnl   EPS_pnl  \
0              0.315280                        0.077313  0.054166   
1              0.315033                        0.079861  0.055763   
2              0.322631                        0.076954  0.054869   
3           

In [123]:
# Save the final DataFrame to a CSV file
final_df.to_csv("preprocessed_financial_data_M.csv", index=False)
print("Preprocessed data saved successfully as 'preprocessed_financial_data_M.csv'.")

Preprocessed data saved successfully as 'preprocessed_financial_data_M.csv'.
